In [ ]:
import requests
import re

# Ask the user for the URL they want to scrape
user_input_url = input("Enter the URL you want to scrape for contact information: ")

try:
    # Send an HTTP GET request to the user-provided URL with SSL certificate verification disabled
    response = requests.get(user_input_url, verify=False)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Use regular expressions to find email addresses, phone numbers, and fax numbers in the HTML content
        email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b')
        phone_pattern = re.compile(r'\b(?:\+\d{1,3}\s?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')
        fax_pattern = re.compile(r'Fax:\s?(\+\d{1,3}\s?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')


        # Find all email addresses, phone numbers, and fax numbers matching the patterns
        contact_info = set(re.findall(email_pattern, response.text))
        contact_info.update(('number phone=' + number for number in re.findall(phone_pattern, response.text)))
        contact_info.update(('number fax=' + number for number in re.findall(fax_pattern, response.text)))

        # Print the found contact information
        for info in contact_info:
            print(info)
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred while making the request: {e}")


In [ ]:
pip install PySimpleGUI

In [ ]:
import requests
import re
import PySimpleGUI as sg

def scrape_website(url):
    try:
        # Send an HTTP GET request to the user-provided URL with SSL certificate verification disabled
        response = requests.get(url, verify=False)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Use regular expressions to find email addresses, phone numbers, and fax numbers in the HTML content
            email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b')
            phone_pattern = re.compile(r'\b(?:\+\d{1,3}\s?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')
            fax_pattern = re.compile(r'Fax:\s?\b(?:\+\d{1,3}\s?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b')

            # Find all email addresses, phone numbers, and fax numbers matching the patterns
            contact_info = set(re.findall(email_pattern, response.text))
            contact_info.update(('number phone=' + number for number in re.findall(phone_pattern, response.text)))
            contact_info.update(('number fax=' + number for number in re.findall(fax_pattern, response.text)))

            # Display the found contact information
            for info in contact_info:
                window['-OUTPUT-'].print(info)
        else:
            window['-OUTPUT-'].print(f"Failed to retrieve the page. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        window['-OUTPUT-'].print(f"An error occurred while making the request: {e}")

# Define the layout
layout = [
    [sg.Text("Enter the URL you want to scrape for contact information:")],
    [sg.InputText(key='-URL-')],
    [sg.Button('Scrape'), sg.Button('Exit')],
    [sg.Output(size=(80, 20), key='-OUTPUT-')],
]

# Create the window
window = sg.Window('Web Scraper Interface', layout)

# Event loop
while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == 'Exit':
        break
    elif event == 'Scrape':
        url = values['-URL-']
        if url:
            scrape_website(url)

# Close the window
window.close()
